`` find . -maxdepth 1 -type f | head -100 | xargs cp -t ../trainable_mp3s``

Run this command in train_mp3s file to copy first 100 files into trainable_mp3s

### extracts the filenames of the mp3 files

Select the names from a the folder where I've kept a subset of the data

In [1]:
import os

# Specify the directory path
# directory_path = "./train_mp3s"  #main dataset directory
directory_path = "./trainable_mp3s"

file_names = []

for filename in os.listdir(directory_path):
    # Get the filename without extension
    file_name, file_extension = os.path.splitext(filename)
    file_names.append(file_name)

print(len(file_names))
# file_names = file_names[0:10000]

100


### Finding out the total duration of the training dataset

In [2]:
import sox
import time
from tqdm import tqdm

pbar2 = tqdm(total=200, position=0)

def sox_length(path):
    try:
        length = sox.file_info.duration(path)
        return length
    except:
        return None

length = 0.0
hrs_save = 0

for i in pbar2(range(len(file_names))):
    length += sox_length(os.path.join(directory_path, f'{file_names[i]}.mp3'))
    if (length /36000) > 1:
        hrs_save = hrs_save + 10
        length = length - 36000
        
hrs = time.strftime("%H", time.gmtime(length))
mins = time.strftime("%M", time.gmtime(length))
secs = time.strftime("%S", time.gmtime(length))
hrs = str(int(hrs) + hrs_save)
print("Total duration :", hrs,"hours",mins, "mins", secs, "secs")

  0%|                                                                                           | 0/200 [00:00<?, ?it/s]

TypeError: 'tqdm' object is not callable

### Play a random audio to check the data quality

In [17]:
from IPython.display import Audio
import random

file_id = random.randint(0, 1000)
audio_file = os.path.join(directory_path, f'{train_path[file_id]}.mp3')

audio = Audio(audio_file)
display(audio)
print(train_text[file_id])

NameError: name 'directory_path' is not defined

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


  _   _          _      _
 | | | |_ __  __| |__ _| |_ ___
 | |_| | '_ \/ _` / _` |  _/ -_)
  \___/| .__/\__,_\__,_|\__\___|
       |_|
                       
Read the migration plan to Notebook 7 to learn about the new features and the actions to take if you are using extensions.

https://jupyter-notebook.readthedocs.io/en/latest/migrate_to_notebook7.html

Please note that updating to Notebook 7 might break some of your extensions.

[I 11:55:14.188 NotebookApp] Loading IPython parallel extension
[I 11:55:14.189 NotebookApp] [jupyter_nbextensions_configurator] enabled 0.6.3
[W 11:55:14.189 NotebookApp] Error loading server extension jupyterlab
    Traceback (most recent call last):
      File "/home/sourove/.local/lib/python3.10/site-packages/notebook/notebookapp.py", line 2047, in init_server_extensions
        mod = importlib.import_module(modulename)
      File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
        return _bootstrap._gcd_import(name[level:]

### Makes a csv file for subset of the data
Opens the main_csv_file and selects the file_rows which are already present in the trainable_mp3s folder and creates the subset_csv_file and writes the rows there

In [2]:
import csv

# Specify the input and output file paths
main_csv_file = 'train.csv'
subset_csv_file = 'train2.csv'


subset_filenames = []
with open(main_csv_file) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        thing_index = file_names.index(row['id']) if row['id'] in file_names else -1
        if thing_index != -1:
            subset_filenames.append(row)
            

with open(subset_csv_file, mode='w', newline='') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=subset_filenames[0].keys())
    csv_writer.writeheader()
    for i in range(len(subset_filenames)):
        csv_writer.writerow(subset_filenames[i])

NameError: name 'file_names' is not defined

In [23]:
# !cat train2.csv | wc -l

In [10]:
import os

def createIfNotExist(path):
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)

### Converting to wav file and txt file from csv and mp3 file into data directory

In [12]:
import os 
import csv

# from multiprocessing.pool import ThreadPool
from sox import Transformer
import tqdm


# data_dir = os.path.join("./", "data2")
# train_dir = os.path.join(data_dir, "train")
# valid_dir = os.path.join(data_dir, "valid")
# createIfNotExist(subset_data_dir)


subset_csv = os.path.join("./", "train2.csv")
mp3_dir = os.path.join("", "./trainable_mp3s")

subset_data_dir = os.path.join("./", "data2")
subset_train_dir = os.path.join(subset_data_dir, "train")
subset_train_wav_dir = os.path.join(subset_train_dir, "wav")
subset_train_txt_dir = os.path.join(subset_train_dir, "txt")

subset_valid_dir = os.path.join(subset_data_dir, "valid")
subset_valid_wav_dir = os.path.join(subset_valid_dir, "wav")
subset_valid_txt_dir = os.path.join(subset_valid_dir, "txt")

createIfNotExist(subset_train_wav_dir)
createIfNotExist(subset_train_txt_dir)

createIfNotExist(subset_valid_wav_dir)
createIfNotExist(subset_valid_txt_dir)

In [26]:
from sox import Transformer
from tqdm import tqdm


def process(x, train_valid_dir):
    wav_dir = os.path.join(train_valid_dir, "wav")
    txt_dir = os.path.join(train_valid_dir, "txt")
    file_name, text, _ = x
#     file_name = os.path.splitext(os.path.basename(file_path))[0]
#     print(file_path, file_name)
    text = text.strip()
    with open(os.path.join(txt_dir, file_name + '.txt'), 'w') as f:
        f.write(text)
    audio_path = os.path.join(mp3_dir, f'{file_name}.mp3')
    output_wav_path = os.path.join(wav_dir, file_name + '.wav')

    tfm = Transformer()
    tfm.rate(samplerate=16000)
    tfm.build(
        input_filepath=audio_path,
        output_filepath=output_wav_path
    )


def convertToWavTxt(csv_file, data_dir):
    train_dir = os.path.join(data_dir, "train")
    valid_dir = os.path.join(data_dir, "valid")
    with open(csv_file) as csvfile:
        
        reader = csv.DictReader(csvfile, delimiter=',')
        data = [(row['id'], row['sentence'], row['split']) for row in reader]
        for i in tqdm(range(len(data))):
            if data[i][2] == 'train':
                process(data[i], train_dir)
            else:
                process(data[i], valid_dir)

In [27]:
convertToWavTxt(subset_csv, subset_data_dir)

NameError: name 'subset_csv' is not defined

### Creating manifest.json file

In [2]:
# data[:50]

In [29]:
# file_paths = []
# with open(csvPath) as csvfile:
#     reader = csv.reader(csvfile)
#     heading = next(reader)
#     for row in reader:
#         file_paths.append(row[0])

### Parse if the wav is for train set or valid set

In [4]:
import csv

def parseTrainValid(filepath):
    train_path = []
    valid_path = []
    with open(filepath) as csvfile:
        spamreader = csv.DictReader(csvfile, delimiter=',')
        for row in spamreader:
            if row['split'] == 'train':
                train_path.append(row['id'])
            else:
                valid_path.append(row['id'])
                
    return (train_path, valid_path)

In [5]:
train_path, valid_path = parseTrainValid('train2.csv')

In [8]:
print(len(valid_path))

3


In [13]:
import json
from pathlib import Path

# data_path = os.path.join(data_dir, "wav")
subset_abs_train = os.path.abspath(subset_train_dir)
subset_abs_valid = os.path.abspath(subset_valid_dir)


# valid_path = os.path.abspath(valid_dir)
# data_path = os.path.abspath(data_dir)


def makeManifest(manifest_path, rootpath:str, files:list):
    
    output_name =  "manifest.json"
    output_path = Path(manifest_path) / output_name
    manifest = {
        'root_path': rootpath,
        'samples': []
    }

    for file_path in tqdm.tqdm(files):
        wav_path = os.path.join(f'wav/{file_path}.wav')
        transcript_path = os.path.join(f'txt/{file_path}.txt')
        manifest['samples'].append({
            'wav_path': wav_path,
            'transcript_path': transcript_path
        })

    output_path.write_text(json.dumps(manifest), encoding='utf8')

In [16]:
makeManifest(subset_train_dir, subset_abs_train, train_path)
makeManifest(subset_valid_dir, subset_abs_valid, valid_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 38597.89it/s]


In [17]:
def _duration_file_path(path):
    return path, sox.file_info.duration(path)


def order_and_prune_files(
        file_paths,
        min_duration,
        max_duration,
        num_workers):
    print("Gathering durations...")
    with Pool(processes=num_workers) as p:
        duration_file_paths = list(tqdm(p.imap(_duration_file_path, file_paths), total=len(file_paths)))
    print("Sorting manifests...")
    if min_duration and max_duration:
        print("Pruning manifests between %d and %d seconds" % (min_duration, max_duration))
        duration_file_paths = [(path, duration) for path, duration in duration_file_paths if
                               min_duration <= duration <= max_duration]

    total_duration = sum([x[1] for x in duration_file_paths])
    print(f"Total duration of split: {total_duration:.4f}s")
    return [x[0] for x in duration_file_paths]  # Remove durations




### Select the data from train.csv and write in train2.csv based on trainable_mp3s folder

In [ ]:
# import
from bnunicodenormalizer import Normalizer 
from pprint import pprint
# initialize
bnorm=Normalizer()
# normalize
word = 'াটোবাকো'
result=bnorm(word)
print(f"Non-norm:{word}; Norm:{result['normalized']}")
print("--------------------------------------------------")
pprint(result)

In [ ]:
# initialize without english (default)
norm=Normalizer()
print("without english:",norm("ASD123")["normalized"])
# --> returns None
norm=Normalizer(allow_english=True)
print("with english:",norm("ASD123")["normalized"])

In [ ]:
transcript = list(filter(None, [labels_map.get(x) for x in list(result['normalized'])]))

In [8]:
import csv

# Specify the input and output file paths
input_csv_file = 'train.csv'
output_csv_file = 'train2.csv'

# Open the input CSV file in read mode
with open(input_csv_file, mode='r', newline='') as input_file:
    # Create a CSV reader
    csv_reader = csv.reader(input_file)

    # Read the header row
    header = next(csv_reader)

    # Read and store the first 1000 rows
    rows_to_write = []
    for i, row in enumerate(csv_reader):
#         if i < 1000:
        rows_to_write.extend(row[1])
#             print(row[1])
#         else:
#             break

# Open the output CSV file in write mode
# with open(output_csv_file, mode='w', newline='') as output_file:
#     # Create a CSV writer
#     csv_writer = csv.writer(output_file)

#     # Write the header row
#     csv_writer.writerow(header)

#     # Write the selected rows
#     csv_writer.writerows(rows_to_write)

print(f"Spliced first 1000 rows from '{input_csv_file}' and wrote to '{output_csv_file}'.")


Spliced first 1000 rows from 'train.csv' and wrote to 'train2.csv'.


In [13]:
len(rows_to_write)
myset = set(rows_to_write)
print(myset)

{'ফ', ' ', 'া', 'ৈ', '৷', 'ঋ', "'", 'খ', 'ঙ', 'উ', '…', '‚', 'ধ', 'ূ', 'য়', ':', ';', 'ক', '?', '–', 'ঠ', 'ও', 'গ', '“', 'থ', 'শ', 'ঁ', 'ে', '—', '॥', 'ৃ', 'ড', 'ঢ়', 'ভ', 'অ', 'ন', 'ৗ', '!', 'ি', 'ঢ', 'ব', 'চ', '”', 'ঘ', 'ষ', '/', '-', 'ঐ', 'স', 'হ', 'আ', 'ত', 'ছ', 'ট', 'ঊ', 'র', 'প', 'ং', '.', '়', 'ঞ', 'ৰ', 'জ', 'ঈ', 'ঃ', 'ণ', '’', 'ৌ', 'এ', 'ল', '"', '‘', 'ু', ',', 'ৎ', '্', 'য', 'দ', 'ঝ', '৵', 'ো', 'ই', 'ঔ', 'ড়', '।', 'ম', 'ী'}


In [15]:
myset

{' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '?',
 '।',
 '॥',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ৗ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৵',
 '৷',
 '–',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '…'}

In [18]:
labels = [
 ' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '?',
 '।',
 '॥',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'ঊ',
 'ঋ',
 'এ',
 'ঐ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঝ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ৗ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 '৵',
 '৷',
 '–',
 '—',
 '‘',
 '’',
 '‚',
 '“',
 '”',
 '…'
]

In [26]:
len(labels)

87

In [23]:
a = '–'
b = '—'

In [25]:
print(ord(a), ord(b))

8211 8212
